In [1]:
import json
import random
from pathlib import Path
import collections

import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
Path.ls = lambda x: list(x.iterdir())
# from urllib.parse import quote_plus

In [2]:
#Select granularity to get the data
REGION_FIELDNAME = 'detecteddistrict' #detecteddistrict - for district; detectedcity - for city

#OUTPUT filename region specifier
REGION_OUTPUT_FILE_REG_SPECCIFIER='district'

#what data to extract
DATA_TYPE = 'Recovered' #'confirmed' - default; 'Hospitalized'; 'Recovered'; 'Deceased'

In [3]:
from datetime import date

today = date.today()

# dd/mm/YY
current_date = today.strftime("%Y%m%d")
print("d1 =", current_date)

d1 = 20200419


In [4]:
import urllib.request, json
from urllib.error import HTTPError

# def get_raw_data(raw_web_url="https://api.covid19india.org/raw_data.json"):
#     with urllib.request.urlopen(raw_web_url) as url:
#         data_dict = json.loads(url.read().decode())
#         return data_dict["raw_data"]


def get_stats_history(stats_history_url="https://api.rootnet.in/covid19-in/stats/history"):
    try:
        with urllib.request.urlopen(stats_history_url) as url:
            data_dict = json.loads(url.read().decode())
            return data_dict
    except HTTPError as e:
        print(f"Using local backup of {stats_history_url}")
        with open("history.json") as f:
            return json.loads(f.read())

def get_stats_history_district(stats_history_url="https://api.covid19india.org/raw_data.json"):
    try:
        with urllib.request.urlopen(stats_history_url) as url:
            data_dict = json.loads(url.read().decode())
            return data_dict
    except HTTPError as e:
        print(f"Using local backup of {stats_history_url}")
        with open("history.json") as f:
            return json.loads(f.read())
        
# def get_state_data(
#     case_count_url="https://api.covid19india.org/state_district_wise.json",
# ):
#     with urllib.request.urlopen(case_count_url) as url:
#         data_dict = json.loads(url.read().decode())
#         return data_dict


# def get_case_count(data, state="Karnataka"):
#     df = pd.DataFrame(data[state]["districtData"])
#     df = df.transpose()
#     df.reset_index(inplace=True)
#     df.rename(columns={"confirmed": "CaseCount", "index": "District"}, inplace=True)
#     df.drop(columns=["lastupdatedtime"], inplace=True)
#     return df

In [5]:
from typing import List, Dict
#stats:List = get_stats_history()["data"]
stats:List = get_stats_history_district()["raw_data"]

In [6]:
if DATA_TYPE == 'confirmed':
    district_df = pd.DataFrame([(i[REGION_FIELDNAME],i['dateannounced']) for i in stats])
else:
    district_df = pd.DataFrame([(i[REGION_FIELDNAME],i['statuschangedate']) for i in stats if i['currentstatus']==DATA_TYPE])
# column names in JHU format
REGION = 'Province/State'
district_df.columns = [REGION,'date']

In [7]:
district_df

,Province/State,date
0,Thrissur,14/02/2020
1,Alappuzha,14/02/2020
2,Kasaragod,14/02/2020
3,East Delhi,15/03/2020
4,Hyderabad,02/03/2020
...,...,...
176,Wayanad,08/04/2020
177,Mysuru,13/04/2020
178,Raipur,31/03/2020
179,Bengaluru,13/04/2020


In [8]:
import pandas as pd
from datetime import datetime
date_list = pd.date_range(start="2020-01-22",end=datetime.today()).strftime("%d/%m/%Y")

In [9]:
district_df_pivot = pd.pivot_table(district_df,values=REGION,index=[REGION],columns=['date'],aggfunc=np.count_nonzero,fill_value=0)
district_df_pivot_dated = district_df_pivot.reindex(date_list, axis=1).fillna(0)
district_df_final = district_df_pivot_dated.cumsum(axis=1)

In [10]:
district_df_final

,22/01/2020,23/01/2020,24/01/2020,25/01/2020,26/01/2020,27/01/2020,28/01/2020,29/01/2020,30/01/2020,31/01/2020,...,10/04/2020,11/04/2020,12/04/2020,13/04/2020,14/04/2020,15/04/2020,16/04/2020,17/04/2020,18/04/2020,19/04/2020
Province/State,,,,,,,,,,,,,,,,,,,,,
Agra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
Ahmadabad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
Alappuzha,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
Bengaluru,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,42.0,42.0,46.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0
Bilaspur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
Chennai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
Chikkaballapura,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,4.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0
Dakshina Kannada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,10.0,12.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0
Davanagere,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0


## Transform to the JHU data format
See this [Github link to data](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv) for reference

In [11]:
district_df_final.insert(0, "Country/Region", "India")
district_df_final.insert(1, "Lat", 20)
district_df_final.insert(2, "Long", 70)

In [12]:
for date in date_list:
    datenew = datetime.strptime(date, "%d/%m/%Y")
    datenew = datetime.strftime(datenew, "%-m/%-d/%y")
    district_df_final = district_df_final.rename(columns = {date:datenew})

In [13]:
district_df_final.head()

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
Province/State,,,,,,,,,,,,,,,,,,,,,
Agra,India,20,70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
Ahmadabad,India,20,70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
Alappuzha,India,20,70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
Bengaluru,India,20,70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,42.0,42.0,46.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0
Bilaspur,India,20,70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [14]:
fname = f'../data/time_series_covid19_{DATA_TYPE}_India_{REGION_OUTPUT_FILE_REG_SPECCIFIER}_{current_date}.csv'
district_df_final.to_csv(fname, header=True)

In [15]:
!cat $fname

Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
Agra,India,20,70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,10.0,10.0,10.0,10.0,10.0,10

In [16]:
fname

'../data/time_series_covid19_Recovered_India_district_20200419.csv'